In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Step 1: Prepare dataset and annotations
# Assume you have prepared your dataset with annotations as bounding box coordinates and class labels.

In [ ]:
class_names = ["Black Rot", "ESCA", "Healthy", "Leaf Blight"]
num_classes = len(class_names)
batch_size = 8

In [ ]:
# Step 2: Load a pre-trained model
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Step 3: Add custom classification and regression heads
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
class_output = Dense(num_classes, activation="softmax", name="class_output")(x)
bbox_output = Dense(num_classes * 4, activation="linear", name="bbox_output")(x)

model = Model(inputs=base_model.input, outputs=[class_output, bbox_output])

In [ ]:
# Step 4: Data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2,
)

train_generator = datagen.flow_from_directory(
    "path_to_train_data",
    target_size=(224, 224),
    batch_size=batch_size,
    subset="training",
)

validation_generator = datagen.flow_from_directory(
    "path_to_train_data",
    target_size=(224, 224),
    batch_size=batch_size,
    subset="validation",
)

In [ ]:
# Step 5: Compile and train the model
model.compile(
    optimizer=Adam(lr=learning_rate),
    loss=["categorical_crossentropy", "mean_squared_error"],
    metrics=["accuracy"],
)

In [ ]:
epochs = 10
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
)

In [ ]:
# Step 6: Evaluate the model
evaluation = model.evaluate(validation_generator, steps=validation_steps_per_epoch)
print("Validation Loss:", evaluation[0])
print("Validation Classification Accuracy:", evaluation[3])
print("Validation Regression Mean Squared Error:", evaluation[4])

In [ ]:
# Step 7: Inference
# Assuming you have a new leaf image for inference
inference_image = load_and_preprocess_image(
    "path_to_inference_image.jpg"
)  # Load and preprocess the image

class_predictions, bbox_predictions = model.predict(
    tf.expand_dims(inference_image, axis=0)
)

In [ ]:
# Step 8: Post-processing
# Example of non-maximum suppression for bounding boxes
def non_max_suppression(boxes, scores, threshold=0.5):
    selected_indices = tf.image.non_max_suppression(
        boxes, scores, max_output_size=10, iou_threshold=threshold
    )
    selected_boxes = tf.gather(boxes, selected_indices)
    selected_scores = tf.gather(scores, selected_indices)
    return selected_boxes, selected_scores


selected_boxes, selected_scores = non_max_suppression(
    bbox_predictions[0], class_predictions[0]
)

# Now you can visualize the selected boxes on the inference image
visualize_inference(
    image_path="path_to_inference_image.jpg",
    boxes=selected_boxes,
    scores=selected_scores,
)